In [1]:
%pwd

'/home/qula0496/quan/Nonstationary_Transformers/ns_models'

In [2]:
%cd ..

/home/qula0496/quan/Nonstationary_Transformers


/home/qula0496/quan/.venv/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [3]:
import os
id_gpu = '1'
n_users = 86
os.environ['CUDA_VISIBLE_DEVICES'] = id_gpu

In [4]:
from utils.data_utils import *
from utils.model_utils import *
# from utils.koopman_utils import *
from modules.serverbase import *
from modules.userbase import *
from modules.servernsTransformer import *
from modules.usernsTranformer import *
import numpy as np
import torch
torch.cuda.init()

In [5]:
num_ts = 862

In [6]:
# Download the dataset
download_traffic_dataset()

# Clean the dataset
traffic_data = clean_traffic(num_ts=num_ts)
traffic_data.shape

Files not exist
/home/qula0496/quan/Nonstationary_Transformers/utils/data_utils.py
data loaded..


(17544, 862)

In [7]:
traffic_data.head()

,MT_001,MT_002,MT_003,MT_004,MT_005,MT_006,MT_007,MT_008,MT_009,MT_010,...,MT_853,MT_854,MT_855,MT_856,MT_857,MT_858,MT_859,MT_860,MT_861,MT_862
time,,,,,,,,,,,,,,,,,,,,,
2015-01-01 00:00:00,0.0048,0.0146,0.0289,0.0142,0.0064,0.0232,0.0162,0.0242,0.0341,0.0375,...,0.0051,0.0051,0.0074,0.0079,0.0051,0.0051,0.0339,0.0051,0.0100,0.0121
2015-01-01 01:00:00,0.0072,0.0148,0.0350,0.0174,0.0084,0.0240,0.0201,0.0338,0.0434,0.0381,...,0.0036,0.0036,0.0107,0.0058,0.0036,0.0036,0.0348,0.0036,0.0087,0.0136
2015-01-01 02:00:00,0.0040,0.0101,0.0267,0.0124,0.0049,0.0170,0.0127,0.0255,0.0332,0.0309,...,0.0030,0.0030,0.0043,0.0050,0.0030,0.0030,0.0327,0.0030,0.0061,0.0107
2015-01-01 03:00:00,0.0039,0.0060,0.0218,0.0090,0.0029,0.0118,0.0088,0.0163,0.0211,0.0199,...,0.0033,0.0033,0.0019,0.0052,0.0033,0.0033,0.0292,0.0033,0.0040,0.0071
2015-01-01 04:00:00,0.0042,0.0055,0.0191,0.0082,0.0024,0.0095,0.0064,0.0087,0.0144,0.0226,...,0.0049,0.0049,0.0011,0.0071,0.0049,0.0049,0.0264,0.0049,0.0040,0.0039


In [8]:
# # prompt: write pandas to csv

# import pandas as pd

# # Save the DataFrame to a CSV file
# traffic_data.to_csv('/home/qula0496/quan/Nonstationary_Transformers/dataset/traffic/traffic.csv', index_label='date')

In [9]:
from ns_models.ns_TransformerConfig import NS_TransformerConfig
user_num_ts = int(traffic_data.shape[1] / n_users)
args = NS_TransformerConfig()
args.devices = id_gpu
args.enc_in = user_num_ts
args.dec_in = user_num_ts
args.c_out = user_num_ts
args.root_path = './dataset/traffic/'
args.data_path = 'traffic.csv'
args.model_id = 'traffic_96_96'
args.data_path

'traffic.csv'

In [10]:
from torch.utils.data import DataLoader
from data_provider.data_factory import *
from data_provider.data_loader import *

test_data, test_loader = data_provider(args, flag='test')

In [11]:
test_data

In [12]:
user_data_list = []
server_data_list = []
for i in range(n_users):
    train_data, train_loader = data_provider(args, flag='train', start=i*user_num_ts+1, end=min(user_num_ts*(i+1)+1, traffic_data.shape[1]+1))
    test_data, test_loader = data_provider(args, flag='test', start=i*user_num_ts+1, end=min(user_num_ts*(i+1)+1, traffic_data.shape[1]+1))
    # user_data = train_set.filter(lambda e, idx: idx>=(i*user_num_ts) and idx < user_num_ts*(i+1), with_indices=True)
    user_data_list.append(train_loader)
    server_data_list.append(test_loader)
    # print(train_data[0][0].shape)

In [13]:
from datetime import datetime
datetime.strptime('2015-01-01 00:00:00', '%Y-%m-%d %H:%M:%S')

datetime.datetime(2015, 1, 1, 0, 0)

In [14]:
from ns_models import ns_Transformer
server_model = ns_Transformer.Model(configs=args)
server_model

Model(
  (enc_embedding): DataEmbedding(
    (value_embedding): TokenEmbedding(
      (tokenConv): Conv1d(10, 64, kernel_size=(3,), stride=(1,), padding=(1,), bias=False, padding_mode=circular)
    )
    (position_embedding): PositionalEmbedding()
    (temporal_embedding): TimeFeatureEmbedding(
      (embed): Linear(in_features=4, out_features=64, bias=False)
    )
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (dec_embedding): DataEmbedding(
    (value_embedding): TokenEmbedding(
      (tokenConv): Conv1d(10, 64, kernel_size=(3,), stride=(1,), padding=(1,), bias=False, padding_mode=circular)
    )
    (position_embedding): PositionalEmbedding()
    (temporal_embedding): TimeFeatureEmbedding(
      (embed): Linear(in_features=4, out_features=64, bias=False)
    )
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): Encoder(
    (attn_layers): ModuleList(
      (0-5): 6 x EncoderLayer(
        (attention): AttentionLayer(
          (inner_attention): DSAttention(
          

In [15]:
import wandb
import torch

NUM_GPUS = torch.cuda.device_count()
LR = 1e-3
GLOBAL_EPOCHS = 30
LOCAL_EPOCHS = 5
BATCH_SIZE = 128
L2_PENALTY = 0.0
USER_RATIO = 0.1
run = wandb.init(
    # project name
    project="Federated Non-Stationary Transformer on Traffic dataset",
    # experiment name
    name=f"Federated Non-Stationary Transformer on Traffic dataset",
    # Hyperparams
    config={
        "dataset": "Traffic860",
        "preprocess_type": "std",
        "num_user": n_users,
        "learning_rate": LR,
        "global_epochs": GLOBAL_EPOCHS,
        "local_epochs": LOCAL_EPOCHS,
        "batch_size": BATCH_SIZE,
        "num_gpus": NUM_GPUS,
        "user_ratio": USER_RATIO,
        "l2_penalty": L2_PENALTY,
        "total_time_series": 860,
        "detrending_data": "No"
    })

config_wanb = wandb.config

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: kenlvq (quanla). Use `wandb login --relogin` to force relogin


In [16]:
# len(server.test_loader[0])

In [17]:
import random
from tqdm import tqdm
from math import sqrt

server = ServerNsTransformer(model=server_model, test_loader=server_data_list)

user_list = []

# Create users
for i in range(config_wanb.num_user):
    user_i = UserNsTransformer(train_loader=user_data_list[i], model=server_model, user_id=i, local_epochs=config_wanb.local_epochs)
    user_list.append(user_i)

for _ in tqdm(range(config_wanb.global_epochs), desc=f"Progress"):
    # Distribute initial model to users
    server.distribute_model(user_list)
    
    # Sub-sample users
    sub_user_list = random.sample(user_list, int(config_wanb.user_ratio * config_wanb.num_user))

    # Check the sub-sampled user and train model
    users_loss = 0.0
    for user in sub_user_list:
        print(user)
        user_loss = user.user_train(args)
        users_loss += user_loss
    # Aggregate weights on server
    server.aggregate_weights(sub_user_list)

    # Calulate avg loss on selected users
    train_loss =  users_loss / len(sub_user_list)

    total_mae = []
    total_mse = []
    for test_loader in server.test_loader:    
        mae, mse, rmse, mape, mspe = server.model_eval(args=args, test_loader=test_loader)
        total_mae.append(mae)
        total_mse.append(mse)
    
    wandb.log({"train_loss": train_loss, "mae": sum(total_mae)/len(total_mae), 'rmse': sqrt(sum(total_mse)/len(total_mse))})

Progress:   0%|          | 0/30 [00:00<?, ?it/s]

Epoch: 1 cost time: 159.44275188446045
Epoch: 2 cost time: 194.96235179901123
Epoch: 3 cost time: 205.79642987251282
Epoch: 4 cost time: 220.32137942314148
Epoch: 5 cost time: 220.21402621269226
Epoch: 1 cost time: 220.50844955444336
Epoch: 2 cost time: 220.42141342163086
Epoch: 3 cost time: 223.3886842727661
Epoch: 4 cost time: 221.83930778503418
Epoch: 5 cost time: 221.2165915966034
Epoch: 1 cost time: 220.9255509376526
Epoch: 2 cost time: 220.80613923072815
Epoch: 3 cost time: 222.60664129257202
Epoch: 4 cost time: 220.60426115989685
Epoch: 5 cost time: 222.14176058769226
Epoch: 1 cost time: 220.81167459487915
Epoch: 2 cost time: 221.64902901649475
Epoch: 3 cost time: 220.55870246887207
Epoch: 4 cost time: 220.96690392494202
Epoch: 5 cost time: 221.97327160835266
Epoch: 1 cost time: 220.70141863822937
Epoch: 2 cost time: 221.21845030784607
Epoch: 3 cost time: 220.9273235797882
Epoch: 4 cost time: 221.9149525165558
Epoch: 5 cost time: 221.57265496253967
Epoch: 1 cost time: 221.243045

Progress:   3%|▎         | 1/30 [2:39:37<77:08:55, 9577.10s/it]

Epoch: 1 cost time: 221.07209730148315
Epoch: 2 cost time: 222.76073670387268
Epoch: 3 cost time: 224.63065457344055
Epoch: 4 cost time: 222.73041486740112
Epoch: 5 cost time: 221.74726843833923
Epoch: 1 cost time: 222.13784670829773
Epoch: 2 cost time: 221.67175555229187
Epoch: 3 cost time: 220.864244222641
Epoch: 4 cost time: 222.54392194747925
Epoch: 5 cost time: 223.6383180618286
Epoch: 1 cost time: 222.996484041214
Epoch: 2 cost time: 223.8771607875824
Epoch: 3 cost time: 223.0657012462616
Epoch: 4 cost time: 221.66459369659424
Epoch: 5 cost time: 222.66889882087708
Epoch: 1 cost time: 222.59312176704407
Epoch: 2 cost time: 222.52136254310608
Epoch: 3 cost time: 221.84228205680847
Epoch: 4 cost time: 221.7093780040741
Epoch: 5 cost time: 224.58722186088562
Epoch: 1 cost time: 221.27929735183716
Epoch: 2 cost time: 222.26824855804443
Epoch: 3 cost time: 222.0439794063568
Epoch: 4 cost time: 222.7795774936676
Epoch: 5 cost time: 221.76563572883606
Epoch: 1 cost time: 222.44388866424

Progress:   7%|▋         | 2/30 [5:21:47<75:11:31, 9667.55s/it]

...

Progress:  93%|█████████▎| 28/30 [39:49:31<56:32, 1696.20s/it]  

Epoch: 1 cost time: 25.640618562698364
Epoch: 2 cost time: 26.01497793197632
Epoch: 3 cost time: 25.851064920425415
Epoch: 4 cost time: 26.121007442474365
Epoch: 5 cost time: 26.19888162612915
Epoch: 1 cost time: 25.425925731658936
Epoch: 2 cost time: 28.885173082351685
Epoch: 3 cost time: 35.008482217788696
Epoch: 4 cost time: 34.46092081069946
Epoch: 5 cost time: 34.089420318603516
Epoch: 1 cost time: 26.105977773666382
Epoch: 2 cost time: 25.55461049079895
Epoch: 3 cost time: 26.547964811325073
Epoch: 4 cost time: 26.119264364242554
Epoch: 5 cost time: 26.37447500228882
Epoch: 1 cost time: 26.01076316833496
Epoch: 2 cost time: 25.14205002784729
Epoch: 3 cost time: 26.064154148101807
Epoch: 4 cost time: 26.331151008605957
Epoch: 5 cost time: 25.738318920135498
Epoch: 1 cost time: 26.21228551864624
Epoch: 2 cost time: 26.593955516815186
Epoch: 3 cost time: 25.642975568771362
Epoch: 4 cost time: 25.531585454940796
Epoch: 5 cost time: 25.601157188415527
Epoch: 1 cost time: 26.4646542072

Progress:  97%|█████████▋| 29/30 [40:10:55<26:12, 1572.58s/it]

Epoch: 1 cost time: 25.90349841117859
Epoch: 2 cost time: 26.08128333091736
Epoch: 3 cost time: 25.924673795700073
Epoch: 4 cost time: 25.627581119537354
Epoch: 5 cost time: 25.480545043945312
Epoch: 1 cost time: 26.653915882110596
Epoch: 2 cost time: 25.6438307762146
Epoch: 3 cost time: 26.41650104522705
Epoch: 4 cost time: 26.00446128845215
Epoch: 5 cost time: 25.367643356323242
Epoch: 1 cost time: 26.005131244659424
Epoch: 2 cost time: 25.422510385513306
Epoch: 3 cost time: 25.576666831970215
Epoch: 4 cost time: 26.597660064697266
Epoch: 5 cost time: 25.946978330612183
Epoch: 1 cost time: 25.44182252883911
Epoch: 2 cost time: 25.726973056793213
Epoch: 3 cost time: 26.32371735572815
Epoch: 4 cost time: 30.987574815750122
Epoch: 5 cost time: 32.43268179893494
Epoch: 1 cost time: 33.00004553794861
Epoch: 2 cost time: 33.76728177070618
Epoch: 3 cost time: 25.610231399536133
Epoch: 4 cost time: 25.888169527053833
Epoch: 5 cost time: 26.280292749404907
Epoch: 1 cost time: 26.1201689243316

Progress: 100%|██████████| 30/30 [40:32:09<00:00, 4864.31s/it]


In [18]:
total_mae

[0.27071172,
 0.2773767,
 0.27653372,
 0.40981764,
 0.2081875,
 0.29567856,
 0.23676822,
 0.2693903,
 0.28507656,
 0.2508345,
 0.30518627,
 0.30213466,
 0.3384769,
 0.22963533,
 0.3547498,
 0.2863843,
 0.38090798,
 0.4227307,
 0.19327697,
 0.28796184,
 0.26567337,
 0.29965132,
 0.23481496,
 0.29147056,
 0.26081344,
 0.28828347,
 0.26662076,
 0.23302324,
 0.26124835,
 0.43700367,
 0.26627868,
 0.23083983,
 0.2964981,
 0.27295944,
 0.24392104,
 0.2441223,
 0.23707978,
 0.27034947,
 0.36346877,
 0.22732395,
 0.2957805,
 0.2692738,
 0.28656518,
 0.23780373,
 0.24926874,
 0.27735156,
 0.3043513,
 0.33343545,
 0.36624274,
 0.45254335,
 0.4220342,
 0.36092323,
 0.35899192,
 0.36593354,
 0.3086495,
 0.256228,
 0.38469914,
 0.37477183,
 0.29514316,
 0.35003108,
 0.35688537,
 0.26829973,
 0.21925047,
 0.34191978,
 0.34482557,
 0.3738855,
 0.46369797,
 0.3413939,
 0.34401873,
 0.40887862,
 0.39626023,
 0.36217025,
 0.3727165,
 0.47654992,
 0.33166164,
 0.3667245,
 0.33637536,
 0.3646787,
 0.42285